In [1]:
import tensorflow.keras as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import cv2

In [2]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [3]:
def convert_to_spectrogram(data, sample_rate):
    return np.array(librosa.feature.melspectrogram(y = data, sr = sample_rate), dtype = "float32")

In [4]:
def power_to_db(S, ref = 1.0, amin = 1e-10, top_db = 80.0):
    S = np.asarray(S)
    if amin <= 0:
        print('ParameterError: amin must be strictly positive')
    if np.issubdtype(S.dtype, np.complexfloating):
        print('Warning: power_to_db was called on complex input so phase '
                      'information will be discarded. To suppress this warning, '
                      'call power_to_db(np.abs(D)**2) instead.')
        magnitude = np.abs(S)
    else:
        magnitude = S
    if callable(ref):
        # User supplied a function to calculate reference power
        ref_value = ref(magnitude)
    else:
        ref_value = np.abs(ref)
    log_spec = 10.0 * np.log10(np.maximum(amin, magnitude))
    log_spec -= 10.0 * np.log10(np.maximum(amin, ref_value))
    if top_db is not None:
        if top_db < 0:
            print('ParameterError: top_db must be non-negative')
        log_spec = np.maximum(log_spec, log_spec.max() - top_db)
    return log_spec

In [5]:
def convert_spectrogram_to_image(spectrogram):
    plt.interactive(False)
    
    figure = plt.figure(figsize = [0.72, 0.72], dpi = 400)
    plt.tight_layout(pad = 0)
    ax = figure.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    
    librosa.display.specshow(power_to_db(spectrogram, ref = np.max))
    
    canvas = FigureCanvas(figure)
    canvas.draw()
    s, (width, height) = canvas.print_to_buffer()

    image = np.fromstring(figure.canvas.tostring_rgb(), dtype = "uint8")
    image = image.reshape((width, height, 3))
    image = cv2.resize(image, (192, 192))

    # Cleaning up the matplotlib instance
    plt.close()    
    figure.clf()
    plt.close(figure)
    plt.close("all")
    
    # Returns a NumPy array containing an image of a spectrogram
    return image

In [6]:
model_uri = "../model/model.h5"

model = load_model(
    model_uri,
    custom_objects = {
        "auc": auc
    }
)

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 64, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 128, 64, 16)       272       
                                                                 
 batch_normalization (BatchN  (None, 128, 64, 16)      64        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 32, 32, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        8224  

In [8]:
model.get_weights()

[array([[[[ 1.86375022e-01,  2.44852036e-01,  3.62003893e-01,
            7.30403364e-02, -6.73651844e-02, -2.19890624e-02,
           -6.72958434e-01,  1.55666694e-01,  3.64349037e-01,
           -2.80455172e-01,  2.20806986e-01, -2.75029182e-01,
           -6.80954233e-02, -2.94041485e-01, -2.41182193e-01,
            4.24467713e-01]],
 
         [[ 2.76921820e-02,  5.21903597e-02,  1.74091920e-01,
           -7.44121373e-02,  6.97799250e-02,  3.49531919e-01,
           -2.82479376e-01, -1.88954756e-01,  4.92364347e-01,
           -5.73497415e-02,  2.32312474e-02, -8.07600245e-02,
           -1.57464772e-01, -2.58396894e-01,  3.14397752e-01,
            3.16280603e-01]],
 
         [[-1.92624301e-01,  8.30447897e-02,  1.80666029e-01,
           -2.46739596e-01,  6.94822893e-02,  2.09835470e-01,
           -1.50687709e-01, -1.35105759e-01,  3.52478325e-01,
            2.43577883e-01,  6.10089637e-02, -2.16170013e-01,
           -3.77860665e-02, -2.16361448e-01,  1.86270967e-01,
      

In [9]:
SAMPLE_RATE_PER_TWO_SECONDS = 44100

In [10]:
sample, sample_rate = librosa.load("../sounds/gunshots/gun01.wav")

In [19]:
spectro = convert_to_spectrogram(sample, sample_rate)

In [16]:
image = convert_spectrogram_to_image(spectro).reshape((-1, 192, 192, 3)).astype(np.float32)
image /= 255

C:\Users\pedro\AppData\Local\Temp/ipykernel_13160/2550250602.py:17: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  image = np.fromstring(figure.canvas.tostring_rgb(), dtype = "uint8")


In [21]:
model.predict(image)

ValueError: in user code:

    File "C:\Users\pedro\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\pedro\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\pedro\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\pedro\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1751, in predict_step
        return self(x, training=False)
    File "C:\Users\pedro\AppData\Roaming\Python\Python39\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\pedro\AppData\Roaming\Python\Python39\site-packages\keras\e